In [1]:
from utils.helper import file_load_large_csv
import matplotlib.pyplot as plt
import numpy as np
import json
import geopandas as gpd
from shapely.geometry import LineString, Point
import plotly.express as px
import pandas as pd
import folium
import ast

In [2]:
df = file_load_large_csv('/home/smebellis/ece5831_final_project/src/processed_data/update_taxi_trajectory1.csv')
df_orig = file_load_large_csv('/home/smebellis/ece5831_final_project/data/train.csv')

In [3]:
# Split the dataframe into smaller ones
# Which makes a list of smaller df
dfs = np.array_split(df, 100)
dfs_org = np.array_split(df, 100)

/home/smebellis/miniconda3/envs/netdetect/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# Make one df to play with
df1 = dfs[0]
df1_orig = dfs_org[0]

In [22]:
central_dispatch = df1.loc[df1.CALL_TYPE=='A']
specific_stand = df1.loc[df1.CALL_TYPE=='B']
random_street = df1.loc[df1.CALL_TYPE=='C']

In [23]:
A_count = central_dispatch.shape[0]
B_count = specific_stand.shape[0]
C_count = random_street.shape[0]

In [ ]:
labels=['taxi central based','taxi stand based','taxi random street']
sizes=[A_count,B_count,C_count]
colors = ['gold', 'yellowgreen', 'lightcoral']
explode = (0.1, 0, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=120)

plt.axis('equal')
plt.show()

In [ ]:
# Define which slice to "explode" by setting the 'pull' manually
# Plotly Express doesn't support 'pull' directly, so we use Plotly Graph Objects for that part
labels=['taxi central based','taxi stand based','taxi random street']
sizes=[A_count,B_count,C_count]
colors = ['gold', 'yellowgreen', 'lightcoral']

df2 = pd.DataFrame({
    'Taxi Type': labels,
    'Count': sizes
})

fig = px.pie(
    df2, 
    names='Taxi Type', 
    values='Count', 
    title='Distribution of Taxi Types',
    color='Taxi Type',
    color_discrete_sequence=colors,
    hole=0.0  # Set to 0 for a standard pie chart; increase for a donut chart
)

# Update traces to add pull
fig.update_traces(
    pull=[0.1, 0, 0],  # Equivalent to explode in Matplotlib
    hoverinfo='label+percent+value',
    textinfo='percent+label',
    textfont_size=14,
    marker=dict(line=dict(color='#FFFFFF', width=2))  # Adds white borders between slices
)

# Customize layout
fig.update_layout(
    title={
        'text': "Distribution of Taxi Types",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    ),
    margin=dict(t=100, l=50, r=50, b=150),
    template='plotly_white'
)

# Display the figure
fig.show()

In [5]:
df1_orig['POLYLINE'] = df1_orig['POLYLINE'].apply(ast.literal_eval)

In [9]:
eval(df1.POLYLINE.iloc[0])[0]

[-8.618643, 41.141412]

In [10]:
df1['TRIP_PATH'] = df1['POLYLINE'].apply(eval)

In [15]:
df1['START_COORDS'] = df1.apply(lambda row: (row['START_LONG'], row['START_LAT']), axis=1)

In [16]:
df1

,TRIP_ID,CALL_TYPE,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,TRAVEL_TIME,POLYLINE_LIST,START,...,START_LONG,START_LAT,END_LONG,END_LAT,WEEKDAY,MONTH,YEAR,TRIP_PATH,NEW,START_COORDS
0,1372636858620000589,C,20000589,2013-07-01 00:00:58,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",7485,"[[-8.618643, 41.141412], [-8.618499, 41.141376...","[-8.618643, 41.141412]",...,-8.618643,41.141412,-8.630838,41.154489,Monday,July,2013,"[[-8.618643, 41.141412], [-8.618499, 41.141376...",[,"(-8.618643, 41.141412)"
1,1372637303620000596,B,20000596,2013-07-01 00:08:23,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",6180,"[[-8.639847, 41.159826], [-8.640351, 41.159871...","[-8.639847, 41.159826]",...,-8.639847,41.159826,-8.665740,41.170671,Monday,July,2013,"[[-8.639847, 41.159826], [-8.640351, 41.159871...",[,"(-8.639847, 41.159826)"
2,1372636951620000320,C,20000320,2013-07-01 00:02:31,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",21105,"[[-8.612964, 41.140359], [-8.613378, 41.14035]...","[-8.612964, 41.140359]",...,-8.612964,41.140359,-8.615970,41.140530,Monday,July,2013,"[[-8.612964, 41.140359], [-8.613378, 41.14035]...",[,"(-8.612964, 41.140359)"
3,1372636854620000520,C,20000520,2013-07-01 00:00:54,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",14055,"[[-8.574678, 41.151951], [-8.574705, 41.151942...","[-8.574678, 41.151951]",...,-8.574678,41.151951,-8.607996,41.142915,Monday,July,2013,"[[-8.574678, 41.151951], [-8.574705, 41.151942...",[,"(-8.574678, 41.151951)"
4,1372637091620000337,C,20000337,2013-07-01 00:04:51,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",9495,"[[-8.645994, 41.18049], [-8.645949, 41.180517]...","[-8.645994, 41.18049]",...,-8.645994,41.180490,-8.687268,41.178087,Monday,July,2013,"[[-8.645994, 41.18049], [-8.645949, 41.180517]...",[,"(-8.645994, 41.18049)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17043,1372946786620000349,C,20000349,2013-07-04 14:06:26,A,False,"[[-8.632332,41.164488],[-8.632386,41.164434],[...",12090,"[[-8.632332, 41.164488], [-8.632386, 41.164434...","[-8.632332, 41.164488]",...,-8.632332,41.164488,-8.609337,41.162571,Thursday,July,2013,"[[-8.632332, 41.164488], [-8.632386, 41.164434...",[,"(-8.632332, 41.164488)"
17044,1372949880620000081,B,20000081,2013-07-04 14:58:00,A,False,"[[-8.628219,41.157954],[-8.627967,41.157783],[...",9525,"[[-8.628219, 41.157954], [-8.627967, 41.157783...","[-8.628219, 41.157954]",...,-8.628219,41.157954,-8.612649,41.169321,Thursday,July,2013,"[[-8.628219, 41.157954], [-8.627967, 41.157783...",[,"(-8.628219, 41.157954)"
17045,1372949214620000518,B,20000518,2013-07-04 14:46:54,A,False,"[[-8.607015,41.150241],[-8.60706,41.150187],[-...",8430,"[[-8.607015, 41.150241], [-8.60706, 41.150187]...","[-8.607015, 41.150241]",...,-8.607015,41.150241,-8.599608,41.153319,Thursday,July,2013,"[[-8.607015, 41.150241], [-8.60706, 41.150187]...",[,"(-8.607015, 41.150241)"
17046,1372946992620000331,A,20000331,2013-07-04 14:09:52,A,False,"[[-8.666676,41.155623],[-8.666658,41.155695],[...",14385,"[[-8.666676, 41.155623], [-8.666658, 41.155695...","[-8.666676, 41.155623]",...,-8.666676,41.155623,-8.584974,41.160735,Thursday,July,2013,"[[-8.666676, 41.155623], [-8.666658, 41.155695...",[,"(-8.666676, 41.155623)"


In [32]:
m = folium.Map(location=[41.155020, -8.630775], zoom_start=15)

kw = {
    "color": "blue",
    "line_cap": "round",
    "fill": True,
    "fill_color": "red",
    "weight": 5,
    
}


folium.Rectangle(
    bounds=[[41.172523, -8.664262], [41.210809, -8.732013]],
    line_join="round",
    dash_array="5, 5",
    **kw,
).add_to(m)




In [33]:
m